In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold,GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

In [2]:
df=pd.read_csv('processed_outliers.csv')
print('Data shape:',df.shape)

Data shape: (1159, 22)


In [3]:
list(enumerate(df.columns))

[(0, 'w/b'),
 (1, 'Water'),
 (2, 'Cement type'),
 (3, 'Cement'),
 (4, 'Slag'),
 (5, 'Fly ash'),
 (6, 'Silica fume'),
 (7, 'Lime filler'),
 (8, 'FA'),
 (9, 'CA'),
 (10, 'Plasticizer'),
 (11, 'Superplasticizer'),
 (12, 'Air entraining'),
 (13, 'Comp. str. test age'),
 (14, 'Compressive strength'),
 (15, 'Air content'),
 (16, 'Spreed'),
 (17, 'Slump'),
 (18, 'Fresh density'),
 (19, 'Dry  density'),
 (20, 'Migration test age'),
 (21, 'Migration resistance')]

1-hot encoding

In [4]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [2])], remainder='passthrough')
X=ct.fit_transform(df)
# remove dummy variable and output
y=X[:,-1]
X=X[:,1:-1]
print(X.shape)
print(y.shape)

(1159, 30)
(1159,)


solver='newton-cg'

In [5]:
param_grid={
    'penalty': [None,'l2'],
    'C': [.1,1,10,100,200]
}
splits=list(StratifiedKFold(shuffle=True,random_state=0).split(X,y))
train_index, test_index = splits[0]

X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

def train_lgreg(X_train,y_train):
    clf = LogisticRegression(multi_class='multinomial',
                            max_iter=int(1e4),
                            solver='newton-cg',
                            random_state=0)
    grid=GridSearchCV(clf,param_grid,cv=5,n_jobs=-1,scoring='accuracy')
    grid.fit(X_train, y_train)
    print(grid.best_params_)
    print(grid.best_score_)
    return grid

def cv_scores(X,y,model):
    acc_test=[]
    acc_train=[]
    for train_index, test_index in StratifiedKFold(shuffle=True,random_state=1).split(X,y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        sc = StandardScaler()
        X_train = sc.fit_transform(X_train)
        X_test = sc.transform(X_test)
        model.fit(X_train,y_train)
        acc_test.append(accuracy_score(y_test,model.predict(X_test)))
        acc_train.append(accuracy_score(y_train,model.predict(X_train)))
    print('Train acc:',np.mean(acc_train))
    print('Test acc:',np.mean(acc_test))

In [6]:
grid=train_lgreg(X_train,y_train)

{'C': 0.1, 'penalty': None}
0.6160244115082825


d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


In [19]:
best_model=LogisticRegression(multi_class='multinomial',max_iter=int(1e4),solver='newton-cg',
                              random_state=0,penalty=None)
best_model.fit(X_train,y_train)
accuracy_score(y_test,best_model.predict(X_test))

0.6724137931034483

In [7]:
best_model=LogisticRegression(multi_class='multinomial',max_iter=int(1e4),solver='newton-cg',
                              random_state=0,penalty=None,C=0.1)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


              precision    recall  f1-score   support

         0.0       0.69      0.74      0.72        46
         1.0       0.61      0.40      0.49        47
         2.0       0.59      0.59      0.59        46
         3.0       0.65      0.70      0.67        47
         4.0       0.78      0.93      0.85        46

    accuracy                           0.67       232
   macro avg       0.66      0.67      0.66       232
weighted avg       0.66      0.67      0.66       232



0.902543497213904

In [20]:
cv_scores(X,y,best_model)

Train acc: 0.7081533868987836
Test acc: 0.6583333333333333


Removing cement type

In [8]:
X=df.drop('Cement type',axis=1).iloc[:,:-1].values
y=df.iloc[:,-1].values
print(X.shape)
print(y.shape)

(1159, 20)
(1159,)


In [ ]:
X_train, X_test = X[train_index], X[test_index]
y_train, y_test = y[train_index], y[test_index]
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
grid=train_lgreg(X_train,y_train)

{'C': 0.1, 'penalty': None}
0.5868875326939842


d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


In [23]:
best_model=LogisticRegression(multi_class='multinomial',max_iter=int(1e4),solver='newton-cg',
                              random_state=0,penalty=None)
best_model.fit(X_train,y_train)
accuracy_score(y_test,best_model.predict(X_test))

0.6637931034482759

In [10]:
best_model=LogisticRegression(multi_class='multinomial',max_iter=int(1e4),solver='newton-cg',
                              random_state=0,penalty=None,C=0.1)
best_model.fit(X_train,y_train)
print(classification_report(y_test,best_model.predict(X_test)))
roc_auc_score(y_test,best_model.predict_proba(X_test),multi_class='ovr')

              precision    recall  f1-score   support

           0       0.69      0.76      0.72        46
           1       0.58      0.38      0.46        47
           2       0.53      0.54      0.54        46
           3       0.69      0.70      0.69        47
           4       0.78      0.93      0.85        46

    accuracy                           0.66       232
   macro avg       0.65      0.66      0.65       232
weighted avg       0.65      0.66      0.65       232



d:\College files\9th sem\mtp\code\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1193: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


0.888238580259111

In [24]:
cv_scores(X,y,best_model)

Train acc: 0.6712664602164937
Test acc: 0.645368711748022
